In [2]:
import cv2
import folium
import numpy as np
import pandas as pd
from folium.plugins import HeatMap
from sklearn.cluster import KMeans


In [3]:
df = pd.read_csv('raw_data.csv', sep=";")

In [4]:
df.columns = ["photo_id", "user_internal_id", "user_name", "lat", "longt", "date_taken", "date_upload",  "visit"]

In [5]:
# Converter a coluna date_taken para o tipo datetime
df['date_taken'] = pd.to_datetime(df['date_taken'])

# Agrupar os dados por user_name e calcular a diferença em dias entre a primeira e a última foto
df['first_date_taken'] = df.groupby('user_name')['date_taken'].transform('min')
df['last_date_taken'] = df.groupby('user_name')['date_taken'].transform('max')
df['days_diff'] = (df['last_date_taken'] - df['first_date_taken']).dt.days

# Marcar os usuários como "Turistas" ou "Local" com base na diferença em dias
df['tipo_usuario'] = 'Indefinido'
df.loc[df['days_diff'] <= 40, 'tipo_usuario'] = 'Turista'
df.loc[df['days_diff'] > 40, 'tipo_usuario'] = 'Local'

# Descartar colunas auxiliares adicionadas
df.drop(['first_date_taken', 'last_date_taken', 'days_diff'], axis=1, inplace=True)

# # Escrever os dados manipulados de volta para o arquivo CSV
# df.to_csv('seu_arquivo_manipulado.csv', index=False)


In [10]:
mapa = folium.Map(location=[-7.144126, -34.857480], zoom_start=12)

# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    # Adicionar um ponto para cada foto no mapa
    cor = 'blue' if row['tipo_usuario'] == 'Local' else "red"
  
    folium.Circle(location=[row['lat'], row['longt']], radius=0.00001, fill=True, color=cor).add_to(mapa)


coordenadas_turistas = df[df['tipo_usuario'] == 'Turista'][['lat', 'longt']].values.tolist()
coordenadas_locais = df[df['tipo_usuario'] == 'Local'][['lat', 'longt']].values.tolist()

# Pré-processamento dos dados: Normalização das coordenadas
df['lat_norm'] = (df['lat'] - df['lat'].mean()) / df['lat'].std()
df['longt_norm'] = (df['longt'] - df['longt'].mean()) / df['longt'].std()

kmeans = KMeans(n_clusters=5)  # Definir o número de clusters desejado
df['cluster'] = kmeans.fit_predict(df[['lat_norm', 'longt_norm']])

for cluster in df['cluster'].unique():
    coordenadas = df[df['cluster'] == cluster][['lat', 'longt']].values.tolist()
    HeatMap(coordenadas, radius=15).add_to(mapa)

# HeatMap(coordenadas_turistas, radius=15).add_to(mapa)
# HeatMap(coordenadas_locais, radius=15).add_to(mapa)
# Salvar o mapa como um arquivo HTML
mapa.save('mapa_fotos.html')

In [21]:
# mapa_densidade2
from sklearn.cluster import DBSCAN


In [25]:
import cv2
import numpy as np

# Carregar a imagem do mapa de calor
mapa_imagem = cv2.imread('mapa_densidade2.png')

# Converter a imagem para o espaço de cores HSV
mapa_hsv = cv2.cvtColor(mapa_imagem, cv2.COLOR_BGR2HSV)

# Definir os intervalos de cor para azul e vermelho no espaço HSV
intervalo_azul = [(100, 150, 50), (140, 255, 255)]  # Azul (H: 100-140)
intervalo_vermelho1 = [(0, 50, 50), (10, 255, 255)]  # Vermelho (H: 0-10)
intervalo_vermelho2 = [(170, 50, 50), (180, 255, 255)]  # Vermelho (H: 170-180)

# Criar máscaras para os pontos azuis e vermelhos
mascara_azul = cv2.inRange(mapa_hsv, *intervalo_azul)
mascara_vermelho1 = cv2.inRange(mapa_hsv, *intervalo_vermelho1)
mascara_vermelho2 = cv2.inRange(mapa_hsv, *intervalo_vermelho2)
mascara_vermelho = cv2.bitwise_or(mascara_vermelho1, mascara_vermelho2)

# Identificar os contornos das áreas azuis e vermelhas
contornos_azul, _ = cv2.findContours(mascara_azul, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contornos_vermelho, _ = cv2.findContours(mascara_vermelho, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos na imagem original
cv2.drawContours(mapa_imagem, contornos_azul, -1, (255, 0, 0), 2)  # Azul
cv2.drawContours(mapa_imagem, contornos_vermelho, -1, (0, 0, 255), 2)  # Vermelho

# Mostrar a imagem com os contornos identificados
cv2.imshow('Contornos Identificados', mapa_imagem)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [33]:
import cv2
import numpy as np

# Carregar a imagem do mapa de calor
mapa_imagem = cv2.imread('mapa_densidade2.png')

# Converter a imagem para o espaço de cores HSV (Matiz, Saturação, Valor)
mapa_hsv = cv2.cvtColor(mapa_imagem, cv2.COLOR_BGR2HSV)

# Definir intervalos de cores para os pontos vermelhos e azuis (no espaço de cores HSV)
intervalo_vermelho_baixo = np.array([0, 100, 100])
intervalo_vermelho_alto = np.array([10, 255, 255])
intervalo_azul_baixo = np.array([110, 100, 100])
intervalo_azul_alto = np.array([130, 255, 255])

# Segmentar a imagem para encontrar áreas vermelhas e azuis
mascara_vermelha = cv2.inRange(mapa_hsv, intervalo_vermelho_baixo, intervalo_vermelho_alto)
mascara_azul = cv2.inRange(mapa_hsv, intervalo_azul_baixo, intervalo_azul_alto)

# Combinação das máscaras para pontos vermelhos e azuis
mascara_combinada = cv2.bitwise_or(mascara_vermelha, mascara_azul)

# Encontrar os contornos das áreas identificadas
contornos, _ = cv2.findContours(mascara_combinada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos na imagem original
cv2.drawContours(mapa_imagem, contornos, -1, (0, 255, 0), 2)

# Mostrar a imagem com os contornos identificados
cv2.imshow('Regiões Populosas', mapa_imagem)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [39]:
import cv2
import numpy as np

# Carregar a imagem do mapa de calor
mapa_imagem = cv2.imread('mapa_densidade2.png')

# Converter a imagem para escala de cinza
mapa_cinza = cv2.cvtColor(mapa_imagem, cv2.COLOR_BGR2GRAY)

# Aplicar um filtro de suavização para reduzir o ruído
mapa_suave = cv2.GaussianBlur(mapa_cinza, (5, 5), 0)

# Aplicar uma transformação de limiarização para identificar áreas densas de pontos
_, mapa_limiarizado = cv2.threshold(mapa_suave, 50, 255, cv2.THRESH_BINARY)

# Encontrar os contornos das áreas densas identificadas
contornos, _ = cv2.findContours(mapa_limiarizado, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos na imagem original
cv2.drawContours(mapa_imagem, contornos, -1, (0, 255, 0), 2)

# Mostrar a imagem com os contornos identificados
cv2.imshow('Regiões Populosas', mapa_imagem)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [45]:
import cv2
import numpy as np

# Carregar a imagem do mapa de calor
mapa_imagem = cv2.imread('mapa_densidade2.png')

# Converter a imagem para escala de cinza
mapa_cinza = cv2.cvtColor(mapa_imagem, cv2.COLOR_BGR2GRAY)

# Aplicar um filtro de suavização para reduzir o ruído
mapa_suave = cv2.GaussianBlur(mapa_cinza, (5, 5), 0)

cv2.imshow("A", mapa_suave)

# Aplicar uma transformação de limiarização para identificar áreas densas de pontos
mapa_limiarizado = cv2.threshold(mapa_suave, 50, 255, cv2.THRESH_BINARY)

# Encontrar os contornos das áreas densas identificadas
contornos, _ = cv2.findContours(mapa_limiarizado, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar os contornos na imagem original
cv2.drawContours(mapa_imagem, contornos, -1, (0, 255, 0), 2)

# Mostrar a imagem com os contornos identificados
cv2.imshow('Regiões Populosas', mapa_imagem)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'findContours'
> Overload resolution failed:
>  - image is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'image'


: 